# Computing a Run Expectancy Matrix

In [1]:
import statcast_pitches
import polars as pl
import plotnine as p9

In [4]:
df = (
    statcast_pitches.load()
    .filter(pl.col('game_date').dt.year() == 2025)
    .with_columns(
        ((pl.col('post_home_score') - pl.col('home_score'))
        + (pl.col('post_away_score') - pl.col('away_score')))
        .alias('delta_runs')
    )
    .select(
        'outs_when_up',
        'balls',
        'strikes',
        'description',
        'events',
        'delta_runs',
    )
    .collect()
)

df.describe()

statistic,outs_when_up,balls,strikes,description,events,delta_runs
str,f64,f64,f64,str,str,f64
"""count""",1.219088e6,1.219088e6,1.219088e6,"""1219088""","""316065""",1.219088e6
"""null_count""",0.0,0.0,0.0,"""0""","""903023""",0.0
"""mean""",0.977837,0.873373,0.897655,null,null,0.030551
"""std""",0.817571,0.966299,0.828895,null,null,0.215727
"""min""",0.0,0.0,0.0,"""ball""","""catcher_interf""",0.0
"""25%""",0.0,0.0,0.0,null,null,0.0
"""50%""",1.0,1.0,1.0,null,null,0.0
"""75%""",2.0,1.0,2.0,null,null,0.0
"""max""",3.0,3.0,3.0,"""swinging_strike_blocked""","""walk""",4.0


In [7]:
df['events'].value_counts().sort(by='count', descending=True)

events,count
str,u32
null,903023
"""field_out""",128457
"""strikeout""",69625
"""single""",45323
"""walk""",25891
…,…
"""catcher_interf""",149
"""sac_fly_double_play""",27
"""triple_play""",6


In [8]:
run_matrix = (
    df.group_by(['outs_when_up', 'balls', 'strikes', 'events'])
    .agg(
        mean_run_exp = pl.col('delta_runs').mean(),
        event_count = pl.col('events').count()
    )
)

run_matrix.sort('mean_run_exp', descending=True)

outs_when_up,balls,strikes,events,mean_run_exp,event_count
i64,i64,i64,str,f64,u32
1,3,1,"""home_run""",1.925532,94
2,1,1,"""home_run""",1.866667,300
2,0,1,"""home_run""",1.86478,318
1,1,2,"""home_run""",1.813953,258
2,1,0,"""home_run""",1.8125,352
…,…,…,…,…,…
2,3,2,"""strikeout""",0.0,4321
2,2,1,"""fielders_choice_out""",0.0,2
0,1,1,"""hit_by_pitch""",0.0,125


In [10]:
# Get detailed event value counts
event_counts = df['events'].value_counts().sort('count', descending=True)
print("Event counts (sorted by frequency):")
print(event_counts)

# Analyze events to suggest binning
def analyze_events_for_binning(events_df):
    """Analyze events and suggest which should be mapped together"""
    
    # Define common event categories
    pitch_events = []
    batted_ball_events = []
    other_events = []
    
    for row in events_df.iter_rows():
        event = str(row[0]).lower() if row[0] is not None else 'none'
        count = row[1]
        
        # Categorize events
        if any(word in event for word in ['ball', 'strike', 'foul', 'hit_by_pitch']):
            pitch_events.append((row[0], count))
        elif any(word in event for word in ['single', 'double', 'triple', 'home_run', 'out', 'error']):
            batted_ball_events.append((row[0], count))
        else:
            other_events.append((row[0], count))
    
    print("\n--- PITCH EVENTS ---")
    for event, count in pitch_events:
        print(f"{event}: {count}")
    
    print("\n--- BATTED BALL EVENTS ---")
    for event, count in batted_ball_events:
        print(f"{event}: {count}")
    
    print("\n--- OTHER EVENTS ---") 
    for event, count in other_events:
        print(f"{event}: {count}")
    
    return pitch_events, batted_ball_events, other_events

pitch_events, batted_ball_events, other_events = analyze_events_for_binning(event_counts)

Event counts (sorted by frequency):
shape: (24, 2)
┌──────────────────────┬────────┐
│ events               ┆ count  │
│ ---                  ┆ ---    │
│ str                  ┆ u32    │
╞══════════════════════╪════════╡
│ null                 ┆ 903023 │
│ field_out            ┆ 128457 │
│ strikeout            ┆ 69625  │
│ single               ┆ 45323  │
│ walk                 ┆ 25891  │
│ …                    ┆ …      │
│ catcher_interf       ┆ 149    │
│ sac_fly_double_play  ┆ 27     │
│ triple_play          ┆ 6      │
│ sac_bunt_double_play ┆ 2      │
│ intent_walk          ┆ 1      │
└──────────────────────┴────────┘

--- PITCH EVENTS ---
strikeout: 69625
hit_by_pitch: 3353
strikeout_double_play: 225

--- BATTED BALL EVENTS ---
field_out: 128457
single: 45323
double: 13534
home_run: 9568
force_out: 5777
grounded_into_double_play: 5465
field_error: 1825
triple: 1146
double_play: 659
fielders_choice_out: 557
sac_fly_double_play: 27
triple_play: 6
sac_bunt_double_play: 2

--- OTHER EV

In [13]:
# Create event mapping based on observed data
event_mapping = {
    # Pitch-level events (most nulls are balls/strikes/fouls)
    None: 'Pitch_Result',  # This represents balls, strikes, fouls - the most common
    
    # Strikeouts
    'strikeout': 'Strikeout',
    'strikeout_double_play': 'Strikeout',
    
    # Walks
    'walk': 'Walk', 
    'intent_walk': 'Walk',
    
    # Hit by pitch
    'hit_by_pitch': 'Hit_By_Pitch',
    
    # Singles
    'single': 'Single',
    
    # Doubles  
    'double': 'Double',
    
    # Triples
    'triple': 'Triple',
    
    # Home runs
    'home_run': 'Home_Run',
    
    # Field outs (combine all types of outs)
    'field_out': 'Field_Out',
    'force_out': 'Field_Out',
    'fielders_choice_out': 'Field_Out',
    
    # Double plays
    'grounded_into_double_play': 'Double_Play',
    'double_play': 'Double_Play',
    'sac_fly_double_play': 'Double_Play', 
    'sac_bunt_double_play': 'Double_Play',
    'triple_play': 'Double_Play',  # Rare, group with double plays
    
    # Sacrifice plays
    'sac_fly': 'Sacrifice',
    'sac_bunt': 'Sacrifice',
    
    # Fielder's choice
    'fielders_choice': 'Fielders_Choice',
    
    # Errors
    'field_error': 'Error',
    
    # Other rare events
    'catcher_interf': 'Other',
    'truncated_pa': 'Other'
}

# Apply the mapping using replace
df = df.with_columns(
    pl.col('events').replace(event_mapping, default='Other').alias('event_bin')
)

# Check the new binned events
binned_counts = df['event_bin'].value_counts().sort('count', descending=True)
print("Binned event counts:")
print(binned_counts)

Binned event counts:
shape: (14, 2)
┌─────────────────┬────────┐
│ event_bin       ┆ count  │
│ ---             ┆ ---    │
│ str             ┆ u32    │
╞═════════════════╪════════╡
│ Pitch_Result    ┆ 903023 │
│ Field_Out       ┆ 134791 │
│ Strikeout       ┆ 69850  │
│ Single          ┆ 45323  │
│ Walk            ┆ 25892  │
│ …               ┆ …      │
│ Sacrifice       ┆ 3194   │
│ Error           ┆ 1825   │
│ Triple          ┆ 1146   │
│ Other           ┆ 735    │
│ Fielders_Choice ┆ 695    │
└─────────────────┴────────┘


/var/folders/my/dqfb5c210fvd90b3sqllxly00000gn/T/ipykernel_98967/2496357364.py:58: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)


In [14]:
# Create run expectancy matrix with binned events
run_matrix_binned = (
    df.group_by(['outs_when_up', 'balls', 'strikes', 'event_bin'])
    .agg(
        mean_run_exp = pl.col('delta_runs').mean(),
        event_count = pl.col('event_bin').count()
    )
)

# Sort by mean run expectancy to see which events/situations produce the most runs
print("Run Expectancy Matrix (Binned Events) - Top 20:")
print(run_matrix_binned.sort('mean_run_exp', descending=True).head(20))

# Also show a summary by event type only (ignoring count)
event_summary = (
    df.group_by(['event_bin'])
    .agg(
        mean_run_exp = pl.col('delta_runs').mean(),
        total_occurrences = pl.col('event_bin').count()
    )
    .sort('mean_run_exp', descending=True)
)

print("\n\nEvent Summary (Average Run Value by Event Type):")
print(event_summary)

Run Expectancy Matrix (Binned Events) - Top 20:
shape: (20, 6)
┌──────────────┬───────┬─────────┬───────────┬──────────────┬─────────────┐
│ outs_when_up ┆ balls ┆ strikes ┆ event_bin ┆ mean_run_exp ┆ event_count │
│ ---          ┆ ---   ┆ ---     ┆ ---       ┆ ---          ┆ ---         │
│ i64          ┆ i64   ┆ i64     ┆ str       ┆ f64          ┆ u32         │
╞══════════════╪═══════╪═════════╪═══════════╪══════════════╪═════════════╡
│ 1            ┆ 3     ┆ 1       ┆ Home_Run  ┆ 1.925532     ┆ 94          │
│ 2            ┆ 1     ┆ 1       ┆ Home_Run  ┆ 1.866667     ┆ 300         │
│ 2            ┆ 0     ┆ 1       ┆ Home_Run  ┆ 1.86478      ┆ 318         │
│ 1            ┆ 1     ┆ 2       ┆ Home_Run  ┆ 1.813953     ┆ 258         │
│ 2            ┆ 1     ┆ 0       ┆ Home_Run  ┆ 1.8125       ┆ 352         │
│ …            ┆ …     ┆ …       ┆ …         ┆ …            ┆ …           │
│ 1            ┆ 2     ┆ 2       ┆ Home_Run  ┆ 1.625407     ┆ 307         │
│ 2            ┆ 3     ┆ 

In [15]:
df.write_csv('run_matrix.csv')